In [1]:
import pysweep
import qcodes as qc
import pysweep.databackends.debug
import pysweep.core.measurementfunctions
from pysweep.core.sweepobject import SweepObject
from pysweep.core.measurementfunctions import MakeMeasurementFunction
import pysweep.convenience as conv
import random
import time

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


In [2]:
station = qc.Station()
pysweep.STATION = station

Define a few parameters to play with

In [3]:
a = qc.instrument.parameter.ManualParameter("a", units="Q")
b = qc.instrument.parameter.ManualParameter("b", units="W")
c = qc.instrument.parameter.ManualParameter("c", units="E")

In [4]:
@MakeMeasurementFunction([['some_value', 'arb.u.']])
def measure(d):
    return [1]

In [5]:
@MakeMeasurementFunction([['Temperature', 'mK']])
def measure_temperature(d):
    return [random.randint(100, 300)/10]

# Simple measurements

This paragraph contains some measurements that corresponds to the easiest cases of measurements. The cases cover 90% of all measurements that will run, but don't show the full power of pysweep

In [6]:
db = pysweep.databackends.debug.DebugDataBackend()
pysweep.databackend = db
pysweep.sweep(lambda d:{}, lambda d:{}, measure, 
              pysweep.sweep_object(a, [0,1,2]),
              pysweep.sweep_object(b, [3,4,5]),
              pysweep.sweep_object(c, [6,7,8]))

<class 'pysweep.databackends.base.DataParameterFixedSweep'> c  numeric True
<class 'pysweep.databackends.base.DataParameterFixedSweep'> b  numeric True
<class 'pysweep.databackends.base.DataParameterFixedSweep'> a  numeric True
<class 'pysweep.databackends.base.DataParameter'> some_value arb.u. numeric False
[('c', 6), ('b', 3), ('a', 0), ('some_value', 1)]
[('c', 6), ('b', 3), ('a', 1), ('some_value', 1)]
[('c', 6), ('b', 3), ('a', 2), ('some_value', 1)]
[('c', 6), ('b', 4), ('a', 0), ('some_value', 1)]
[('c', 6), ('b', 4), ('a', 1), ('some_value', 1)]
[('c', 6), ('b', 4), ('a', 2), ('some_value', 1)]
[('c', 6), ('b', 5), ('a', 0), ('some_value', 1)]
[('c', 6), ('b', 5), ('a', 1), ('some_value', 1)]
[('c', 6), ('b', 5), ('a', 2), ('some_value', 1)]
[('c', 7), ('b', 3), ('a', 0), ('some_value', 1)]
[('c', 7), ('b', 3), ('a', 1), ('some_value', 1)]
[('c', 7), ('b', 3), ('a', 2), ('some_value', 1)]
[('c', 7), ('b', 4), ('a', 0), ('some_value', 1)]
[('c', 7), ('b', 4), ('a', 1), ('some_va

# Extended examples

Here are a few examples of measurements that are still relatively simples, but with a few more bells and whistles

say that we want to execute some function after we set the b parameter

In [7]:
db = pysweep.databackends.debug.DebugDataBackend()
pysweep.databackend = db
pysweep.sweep(lambda d:{}, lambda d:{}, measure, 
              pysweep.sweep_object(a, [0,1,2]),
              conv.add_function(pysweep.sweep_object(b, [3,4,5]), measure_temperature),
              pysweep.sweep_object(c, [6,7,8]))

<class 'pysweep.databackends.base.DataParameterFixedSweep'> c  numeric True
<class 'pysweep.databackends.base.DataParameterFixedSweep'> b  numeric True
<class 'pysweep.databackends.base.DataParameter'> Temperature mK numeric False
<class 'pysweep.databackends.base.DataParameterFixedSweep'> a  numeric True
<class 'pysweep.databackends.base.DataParameter'> some_value arb.u. numeric False
[('c', 6), ('b', 3), ('Temperature', 17.2), ('a', 0), ('some_value', 1)]
[('c', 6), ('b', 3), ('Temperature', 17.2), ('a', 1), ('some_value', 1)]
[('c', 6), ('b', 3), ('Temperature', 17.2), ('a', 2), ('some_value', 1)]
[('c', 6), ('b', 4), ('Temperature', 20.7), ('a', 0), ('some_value', 1)]
[('c', 6), ('b', 4), ('Temperature', 20.7), ('a', 1), ('some_value', 1)]
[('c', 6), ('b', 4), ('Temperature', 20.7), ('a', 2), ('some_value', 1)]
[('c', 6), ('b', 5), ('Temperature', 17.8), ('a', 0), ('some_value', 1)]
[('c', 6), ('b', 5), ('Temperature', 17.8), ('a', 1), ('some_value', 1)]
[('c', 6), ('b', 5), ('Temp

# Adaptive sweep

In [8]:
@MakeMeasurementFunction([['timestamp', 's']])
def smart_point_function(dict_waterfall):
    timestamp = int(time.time())
    points = [int(c) for c in str(timestamp)]
    return points, [timestamp]

In [9]:
db = pysweep.databackends.debug.DebugDataBackend()
pysweep.databackend = db
pysweep.sweep(lambda d:{}, lambda d:{}, measure, 
              pysweep.sweep_object(a, [0,1]),
              conv.add_function(pysweep.sweep_object(b, [3,4]), measure_temperature),
              pysweep.sweep_object(c, smart_point_function))

<class 'pysweep.databackends.base.DataParameter'> timestamp s numeric False
<class 'pysweep.databackends.base.DataParameterFixedSweep'> c  numeric True
<class 'pysweep.databackends.base.DataParameterFixedSweep'> b  numeric True
<class 'pysweep.databackends.base.DataParameter'> Temperature mK numeric False
<class 'pysweep.databackends.base.DataParameterFixedSweep'> a  numeric True
<class 'pysweep.databackends.base.DataParameter'> some_value arb.u. numeric False
[('timestamp', 1568208705), ('c', 1), ('b', 3), ('Temperature', 24.8), ('a', 0), ('some_value', 1)]
[('timestamp', 1568208705), ('c', 1), ('b', 3), ('Temperature', 24.8), ('a', 1), ('some_value', 1)]
[('timestamp', 1568208705), ('c', 1), ('b', 4), ('Temperature', 28.0), ('a', 0), ('some_value', 1)]
[('timestamp', 1568208705), ('c', 1), ('b', 4), ('Temperature', 28.0), ('a', 1), ('some_value', 1)]
[('timestamp', 1568208705), ('c', 5), ('b', 3), ('Temperature', 13.6), ('a', 0), ('some_value', 1)]
[('timestamp', 1568208705), ('c', 5